In [2]:
input_file = 'file_in.csv'
output_file = 'file_out.csv'
import datetime
import csv
from datetime import timedelta

#Sql 
import sqlite3
con = sqlite3.connect('KomunikacjaMiejska.db')
cur = con.cursor()

#Funkcja do sprawdzenia poprawności daty Dzień/Miesiąc/Rok
def SprawdzenieDaty(inputDate):
    try:
        day, month, year = inputDate.split('/')
        datetime.datetime(int(year), int(month), int(day))
        return True
    except ValueError:
        return False

#Funkcja do sprawdzenia poprawności czasu
def SprawdzenieCzasu(inputTime):
    try:
        hour, minute = inputTime.split(':')
        datetime.time(int(hour), int(minute))
        return True
    except ValueError:
        return False
    
def convertTime(date_time):
    format = '%H:%M' # The format
    datetime_str = datetime.datetime.strptime(date_time, format)
   
    return datetime_str

def opoznieniePrzyjazdu(czas1, czas2):
    if(czas2>czas1):
        czasout = czas2 - czas1
    else:
        czasout = datetime.datetime.strptime('00:00', '%H:%M')
        czasout = czasout - czasout
    return czasout

def sprawdzenieCzyPoPolnocy(czas):
    if czas.days < 0:
        czas = timedelta(
            days=0,
            seconds=czas.seconds,
            microseconds=czas.microseconds
        )
        
    return czas
        


#Otworzenie pliku file_in jako tylko do odczytu, oraz otworzenie file_out jako pliku tylko do zapisu.
with open(input_file, 'r') as in_csv, open(output_file , mode='w') as out_csv:
    reader = csv.reader(in_csv, delimiter=',')
    fieldnames = ['Nr_przystanku','Nr_lini','Nr_pojazdu','data','Godzina_planowana_przyjazdu','Godzina_faktyczna_przyjazdu','Godzina_faktyczna_odjazdu']
    writer = csv.DictWriter(out_csv, fieldnames=fieldnames)
    line_count = 0
    pop_count = 0
    niepop_count = 0
    try:
        cur.execute('''CREATE TABLE KomunikacjaMiejska(Nr_przystanku int, Nr_lini int, Nr_pojazdu int, data text, Godzina_planowana_przyjazdu text, Godzina_faktyczna_przyjazdu time, Godzina_faktyczna_odjazdu time,wartosc_opoźnienia_odjazdu text,Jak_dlugo_na_przystanku text)''')
    except:
        print(f"Tabela istnieje")
        pass
    
    for row in reader:
        #Jeśli jest to pierwsza linia.
        if line_count == 0:
            #Wczytanie pierwszego rzędu i nadanie go do output pliku
            #print(f'Column names are {" ".join(row)}')
            line_count += 1
        else:
            if((row[0].isdigit() and row[1].isdigit() and row[2].isdigit() and SprawdzenieDaty(row[3]) and SprawdzenieCzasu(row[4]) and SprawdzenieCzasu(row[5]) and SprawdzenieCzasu(row[6]))):
                #Wpisanie dobrych danych do SQLa
                opoznienieczas = str(opoznieniePrzyjazdu(convertTime(row[4]),convertTime(row[5])))

                ilenaprzystanku = str(sprawdzenieCzyPoPolnocy(convertTime(row[6])-convertTime(row[5])))

                cur.execute("INSERT INTO KomunikacjaMiejska VALUES (?,?,?,?,?,?,?,?,?)",(row[0],row[1],row[2],row[3],row[4],row[5],row[6],opoznienieczas,ilenaprzystanku))
                con.commit()
                pop_count += 1
            else:
                #Wpisanie złych danych do pliku
                writer.writerow({'Nr_przystanku':row[0],'Nr_lini':row[1],'Nr_pojazdu':row[2],'data':row[3],'Godzina_planowana_przyjazdu':row[4],'Godzina_faktyczna_przyjazdu':row[5],'Godzina_faktyczna_odjazdu':row[6]})
                niepop_count += 1

            line_count += 1
    print(f'Przetworzono {line_count-1} lini.')
    print(f'Przetworzono Poprawnych {pop_count} lini.')
    print(f'Przetworzono Niepoprawnych {niepop_count} lini.')
    #cur.execute("SELECT * FROM KomunikacjaMiejska")
    #myresult = cur.fetchall()

    #for x in myresult:
      #  print(x)
    #con.commit()
    con.close()




Tabela istnieje
Przetworzono 100 lini.
Przetworzono Poprawnych 93 lini.
Przetworzono Niepoprawnych 7 lini.
